# Scaling Dask in Kubernetes

### Table of Contents
1. Dask overview
2. Kubernetes overview
3. Overview of installing Dask
    1. Installation using Helm
    2. Other installation methods
4. Overview of integrating Dask and Kubernetes
    1. Scaling up/down cluster using dask_kubernetes
5. Next Steps

## Dask Overview

Dask is a flexible library for parallel computing in Python. It is purpose-built to parallelize python data science applications from a single laptop all the way up  to a complex 100+ node cluster.

It is composed of a Dask scheduler and a scaling number of Dask workers and the APIs are designed to be familiar for anyone who has used Pandas or Numpy in the past.

By itself Dask accelerates many machine learning applications, but when paired with the additional acclerations of GPUs integrated through the RAPIDS modules it becomes a very powerful tool that no data scientist should be without. 

For more information about dask see [here](https://docs.dask.org/en/latest/)

## Kubernetes
Kubernetes (or K8S) is an open source tool for managing container workloads and services. K8S is designed to scale, and can run on single node systems all the way up to entire clouds.

K8S allows you to deploy docker containers to run tasks. These docker containers are deployed in pods, which can have resources limitations defined, execution commands set, and allows you to specify custom docker images.

K8S allows dynamic resources addition/removal and can be run on-prem, in the cloud, or using hybrid models.

For more information about Kubernetes see [here](https://kubernetes.io/docs/home/)

## Dask and Kubernetes Integration

By combining the cluster-management and auto-scaling capabilities of Kubernetes with the parallel computing and distributed resource management capabilities of dask we can create a data science environment that dynamically determines resources needs, grows to meets those needs, and optimally executes all data processesing, training, and inferencing in our cluster.

## Overview of installing Dask


### Installation using Helm


In [ ]:
# XXX: These must  be run on the K8S managment server, not through this pod.
# Additional steps can be found in the DeepOps project here: https://github.com/NVIDIA/deepops/blob/master/docs/rapids-dask.md
!helm install -n rapids --namespace rapids --values helm/rapids.yml stable/dask
!kubectl create -f k8s/roles.yaml

Dask can also be installed by using a series of K8S YAML files or by manually deploying and configuring pods. There is more information regarding these methods available [here](http://kubernetes.dask.org/en/latest/).

## Overview of integrating Dask and Kubernetes



### Scaling up/down cluster using dask_kubernetes


#### Manual
First we will need to create a client that will connect to the existing Dask scheduler.

We can see the current resources available, the number of workers, and links to the scheduler and dashboard. Note that the links may not be valid depending on your environment and you may need to use the IPs and ports designated by the cluster admin.

In [ ]:
import dask_kubernetes as dk
from dask.distributed import Client

client = Client()
client

Now we will define a specification for our workers. It is best practice to use the same Docker image for the workers as this notebook. It is also necessary to set the dask-worker resources args to match the K8S resources. Otherwise K8S might kill run Dask jobs.

We pull the scheduler URL out of the client object and write it to our yaml file so that the worker knows who to communicate with.

In [ ]:
worker_spec_fname = '/worker_spec.yaml'
worker_spec = '''
# worker-spec.yml

kind: Pod
metadata:
  labels:
    foo: bar
spec:
  restartPolicy: Never
  containers:
  - image: supertetelman/k8s-rapids-dask:cuda9.2-runtime-ubuntu16.04
    imagePullPolicy: IfNotPresent
    args: [dask-worker, {}, --nthreads, '1', --no-bokeh, --memory-limit, 6GB, --no-bokeh, --death-timeout, '60']
    name: dask
    resources:
      limits:
        cpu: "2"
        memory: 6G
        nvidia.com/gpu: 0
      requests:
        cpu: "2"
        memory: 6G
        nvidia.com/gpu: 0
'''.format(client._start_arg) # Note that we are telling the worker to communicate with the scheduler that has already been launched

with open(worker_spec_fname, "w") as yaml_file:
    yaml_file.write(worker_spec)

We now define the Dask Kubernetes worker cluster. After this all Dask workers launched will match the spec we just defined. As a best practice, all workers in the cluster should be identical.

In [ ]:
cluster = dk.KubeCluster.from_yaml(worker_spec_fname)

We should now see that the cluster has not been modified yet.

In [ ]:
client

We can now manually scale up the cluster to contain N workers.

This can take some time to complete. If the `client` command output does not change try waiting 30 seconds and running it again.

Be sure not to launch more workers than you have available resources in your cluster. Doing this may require manually deleting pods through the Kubernetes interface.

In [ ]:
cluster.scale(1)

In [ ]:
client

And we can scale up some more.

In [ ]:
cluster.scale(2)

In [ ]:
client

And then we can scale down.

In [ ]:
cluster.scale(0)

In [ ]:
client

## Next Steps

Now that we know the basics of installing Dask into K8S, defining worker nodes, and scaling a Dask cluster we can build a machine learning pipeline to take advantage of the parallelism. 

In the next notebook we'll take a look at creating a single worker Dask workload and then see how easy it is to accelerate via scaling in Kubernetes.

After using parallesim to scale across nodes we will integrate the RAPIDS libary and accelerate the application further using GPU enabled machine learning libraries.

Later, we'll also touch upon best-practices for sharing large volumes and storage across your Kubernetes cluster for Dask to use.